## 1 
Подготовить обучающую выборку, каждый элемент которой будет состоять из пар (X, D)m (m=1,…q) – обучающего вектора X = (x1,…,xn) (i=1,…,n) с вектором желаемых значений D = (d1,…,dk) (j=1,…,k) выходов персептрона.

In [1]:
from datareader import DataReader

train_dir = "data/train"
test_dir = "data/test"

input_height = 28
input_width = 28

input_channels = 1
num_classes = 6

train_generator = DataReader(train_dir, [input_height, input_width], True, input_channels, num_classes).get_generator()
test_generator = DataReader(test_dir, [input_height, input_width], False, input_channels, num_classes).get_generator()


print('Size of training set: {}'.format(train_generator.get_data_size()))
print('Size of testing set: {}'.format(test_generator.get_data_size()))

Size of training set: 24
Size of testing set: 6


## 2

Генератором случайных чисел всем синаптическим весам wij и нейронным смещениям w0,j (i=1,…,n; j=1,…,k) присваиваются некоторые малые случайные значения.


In [2]:
from pure_python.neuron import Neuron
from random import random
from pprint import pprint

inputs_count = input_height * input_width

range_min = - 0.0003
range_max = 0.0003


def generate_weights():
    weights = []
    for i in range(inputs_count+1):
        weights.append(range_min + (range_max - range_min) * random())
    return weights
    

pprint(generate_weights())

[0.00019641575637092268,
 0.00026497909051033207,
 0.0002596680538892473,
 -0.00016540158619136574,
 8.713249459581929e-05,
 0.000258545853333994,
 -0.00012032232135914518,
 6.736956502693508e-05,
 -0.00017916254333099005,
 5.884856605564546e-05,
 9.210350978318806e-05,
 -0.00015792912778896817,
 -0.00023469767171638859,
 -0.00012211027454046768,
 -6.103952155878882e-05,
 7.54040954628781e-05,
 -0.00023064922206980268,
 -5.7843897509448266e-05,
 -7.643676528059037e-05,
 -0.00016094372361038367,
 -6.187748053008342e-05,
 0.00029145388108542776,
 -0.000115753410580572,
 -5.977797846165974e-05,
 -6.864892023081262e-05,
 -0.00010634489235660557,
 -7.59532601344812e-05,
 -0.00016975720876022588,
 0.000261997923775612,
 -0.0002333841310418182,
 -0.00016549299018325575,
 0.00014203612455782874,
 5.4169187025831795e-05,
 -0.00012431310631308336,
 -0.00012061566858603172,
 -1.3031970218937029e-05,
 0.0002451225373047466,
 7.599007997947677e-05,
 -0.0002834402327625615,
 0.00014645342457470705,


## 3
Из обучающей выборки (X, D)1,…,(X, D)q, взять следующий по счету вектор Xт = (x1,…,xn) и подать его на входы персептрона x1,…,xn. Сигналам нейронных входов смещения x0 присваиваются единичные значения: x0 = 1.

In [3]:
from pure_python._vector import Vector

x, d = train_generator.next()
vector = Vector(x, d)

neurons = [Neuron(inputs_count) for i in range(num_classes)]

for neuron in neurons:
    neuron.set_weights(generate_weights())

pprint(neurons)

## 4
Шаг 4. Для каждого j-го нейрона вычислить взвешенную сумму входных сигналов netj и выходной сигнал yj на основании функции активации f:
netj=i=0nwi,jxi
yj=fnetj


In [4]:
for neuron in neurons:
    neuron.calc_y(vector.get_x())
    print(f"y: {neuron.get_y()}")
    print(f"net: {neuron.get_net()}")

y: 0.5273918789440062
net: 0.10967732712623107
y: 0.6179639149287819
net: 0.48091499798765025
y: 0.250306072316556
net: -1.0969805685167797
y: 0.5250418386623359
net: 0.10025123334563789
y: 0.6730031045272581
net: 0.7217992657167469
y: 0.5434270516126177
net: 0.17414699130534347


## 5
Для каждого j-го нейрона вычислить его ошибку

In [5]:

for j, neuron in enumerate(neurons):
    loss = pow(vector.get_d()[j] - neuron.get_y(), 2)
    print(f"{j}: loss = {loss}")

0: loss = 0.2781421939760893
1: loss = 0.38187940015410676
2: loss = 0.06265312983854096
3: loss = 0.22558525502125457
4: loss = 0.4529331787033275
5: loss = 0.2953129604243827


## 6
Произвести коррекцию синаптических весов j-го нейрона и нейронных смещений

In [6]:
learning_rate = 1e-6
weights_deltas = [[0] * (len(vector.get_x()) + 1)] * len(neurons)

for j, neuron in enumerate(neurons):
    sigma = (vector.get_d()[j] - neuron.get_y()) * neuron.derivative()
    weights_deltas[j][0] = learning_rate * sigma
    wlen = len(neuron.get_weights())
    for i in range(wlen):
        weights_deltas[j][i] = learning_rate * sigma * vector.get_x()[i]
    neuron.correct_weights(weights_deltas[j])

## 7
Шаги 4-6 повторяются для всех нейронов персептронного слоя при подаче конкретного образа.

Шаги 3-6 выполняются последовательно для каждого входного образа, на котором обучается персептрон.

Шаг 7. После подачи последнего обучающего вектора, проверить критерий останова обучения, если он выполняется, то завершить обучение. В противном случае – возврат к шагу 3.

In [7]:
train_generator.reset_pointer()
train_generator.shuffle_data()

num_epochs = 100

neurons = [Neuron(inputs_count) for i in range(num_classes)]

for neuron in neurons:
    neuron.set_weights(generate_weights())

    
neu_loss = 0
    
for epoch in range(num_epochs):
    neu_loss = 0
    x, d = train_generator.next()
    vector = Vector(x, d)
    
    for neuron in neurons:
        neuron.calc_y(vector.get_x())
    
    weights_deltas = [[0] * (len(vector.get_x()) + 1)] * len(neurons)
    
    for j, neuron in enumerate(neurons):
        sigma = (vector.get_d()[j] - neuron.get_y()) * neuron.derivative()
        weights_deltas[j][0] = learning_rate * sigma
        wlen = len(neuron.get_weights())
        for i in range(wlen):
            weights_deltas[j][i] = learning_rate * sigma * vector.get_x()[i]
        neuron.correct_weights(weights_deltas[j])
    
    loss = 0
    for j, neuron in enumerate(neurons):
        loss += pow(vector.get_d()[j] - neuron.get_y(), 2)
    
    
    neu_loss += 0.5 * loss
    train_generator.reset_pointer()
    train_generator.shuffle_data()
    
    print(f"epoch: {epoch+1}, loss: {neu_loss}")

epoch: 1, loss: 0.9038848130406139
epoch: 2, loss: 0.7881002483161422
epoch: 3, loss: 0.6362610009028915
epoch: 4, loss: 0.2848049833828661
epoch: 5, loss: 0.4954052087642717
epoch: 6, loss: 0.5866818206155423
epoch: 7, loss: 0.23997442972122263
epoch: 8, loss: 0.7159227584713436
epoch: 9, loss: 0.4887555149410579
epoch: 10, loss: 0.4812871055183363
epoch: 11, loss: 0.2587740567711176
epoch: 12, loss: 0.26754971785825243
epoch: 13, loss: 0.26357024852468436
epoch: 14, loss: 0.4844288547898613
epoch: 15, loss: 0.5088751160361137
epoch: 16, loss: 0.1500160410066824
epoch: 17, loss: 0.2734986248139541
epoch: 18, loss: 0.446931730053779
epoch: 19, loss: 0.25632055253034997
epoch: 20, loss: 0.2434449766686851
epoch: 21, loss: 0.07788024346867888
epoch: 22, loss: 0.3829342939457583
epoch: 23, loss: 0.29636803920508104
epoch: 24, loss: 0.06680009181986575
epoch: 25, loss: 0.4172596821997218
epoch: 26, loss: 0.2645830228551617
epoch: 27, loss: 0.25194259091565624
epoch: 28, loss: 0.44341538818

## 8
Test

In [8]:
def get_max_neuron_idx(neurons):
    max_idx = -1
    answer = -1
    for j in range(len(neurons)):
        if neurons[j] > answer:
            answer = neurons[j]
            max_idx = j
    return max_idx

for i in range(test_generator.get_data_size()):
    x, d = test_generator.next()
    vector = Vector(x, d)
    y = [0] * len(neurons)
    for j, neuron in enumerate(neurons):
        neuron.calc_y(vector.get_x())
        y[j] = neuron.get_y()

    print(f"{get_max_neuron_idx(d)} recognized as {get_max_neuron_idx(y)}")

0 recognized as 0
1 recognized as 1
2 recognized as 2
3 recognized as 3
4 recognized as 4
5 recognized as 5
